In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import time
import re
from langdetect import detect

from collections import Counter

import gc

In [2]:
FILENAME = "data/train_series.csv"
FILENAME_ECB = "data/ecb_data.csv"
FILENAME_FED = "data/fed_data.csv"

In [3]:
ecb = pd.read_csv(FILENAME_ECB, index_col=0)
fed = pd.read_csv(FILENAME_FED, index_col=0)

In [4]:
train_series = pd.read_csv(FILENAME, index_col=0)
train_series.head(2)

,Index Name,Index - 9,Index - 8,Index - 7,Index - 6,Index - 5,Index - 4,Index - 3,Index - 2,Index - 1,Index - 0,index ecb,index fed,Index + 1
0,V2X Index,0.001045,0.005841,0.003832,-0.027519,-0.103565,-0.045086,-0.011265,0.005164,0.054050,0.015779,"1126,1471",96,0.010597
1,MOVE Index,-0.021497,0.007891,-0.013175,-0.008436,0.000000,0.026303,0.000556,0.001455,0.007422,0.000000,975,568,0.014492


In [5]:
ecb.head(2)

,title,speaker,text
0,Comments by Yves Mersch at Financial Services ...,Yves Mersch,Comments by Yves Mersch at Financial Service...
1,Securing sustained economic growth in the euro...,Vítor Constâncio,Securing sustained economic growth in the eu...


In [6]:
fed.head(2)

,title,speaker,text
0,The Importance of Economic Education and Finan...,Governor Frederic S. Mishkin,As ...
1,Financial Innovation and Consumer Protection,Chairman Ben S. Bernanke,"The concept of financial innovation, it seems..."


In [7]:
ecb.isna().sum()

title       0
speaker     1
text       34
dtype: int64

In [8]:
embeddings_dict = {}
with open("../glove/glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [14]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: np.linalg.norm(embeddings_dict[word] - embedding))

In [18]:
embeddings_dict['king']

array([ 0.50451 ,  0.68607 , -0.59517 , -0.022801,  0.60046 , -0.13498 ,
       -0.08813 ,  0.47377 , -0.61798 , -0.31012 , -0.076666,  1.493   ,
       -0.034189, -0.98173 ,  0.68229 ,  0.81722 , -0.51874 , -0.31503 ,
       -0.55809 ,  0.66421 ,  0.1961  , -0.13495 , -0.11476 , -0.30344 ,
        0.41177 , -2.223   , -1.0756  , -1.0783  , -0.34354 ,  0.33505 ,
        1.9927  , -0.04234 , -0.64319 ,  0.71125 ,  0.49159 ,  0.16754 ,
        0.34344 , -0.25663 , -0.8523  ,  0.1661  ,  0.40102 ,  1.1685  ,
       -1.0137  , -0.21585 , -0.15155 ,  0.78321 , -0.91241 , -1.6106  ,
       -0.64426 , -0.51042 ], dtype=float32)

In [35]:
ecb[~ecb['text'].isna()].size

5214

In [38]:
ecb[ecb['text'].isna()].size

102

In [39]:
ecb['text'].isna().sum()

34